# IMPORT LIBRARIES

In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

# IMPORT DATASETS

In [2]:
train = pd.read_csv("Input/train.csv", index_col=0)
predict = pd.read_csv("Input/predict.csv", index_col=0)

# PREPARE DATASETS

In [3]:
le = preprocessing.LabelEncoder()
train["cut_num"] = le.fit_transform(train["cut"])
train["color_num"] = le.fit_transform(train["color"])
train["clarity_num"] = le.fit_transform(train["clarity"])

In [4]:
columnas = [a for a in train.columns if a not in ["price","cut","color","clarity"]]
X_train = train[columnas]
y_train = train["price"]

In [5]:
predict["cut_num"] = le.fit_transform(predict["cut"])
predict["color_num"] = le.fit_transform(predict["color"])
predict["clarity_num"] = le.fit_transform(predict["clarity"])

In [6]:
columnas1 = [a for a in predict.columns if a not in ["cut","color","clarity"]]
X_test = predict[columnas]

# TRAIN MODEL

In [7]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(X_train,y_train)

GradientBoostingRegressor()

# OBTAIN RESULT PREDICTION

In [8]:
y_pred = model.predict(X_test)

# TRANSFORM IT TO A DATAFRAME AND EXPORT IT TO DISK AS A CSV

In [9]:
X_test["price"] = y_pred
X_test["id"] = X_test.index

<ipython-input-9-e0da09bb2e45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["price"] = y_pred
<ipython-input-9-e0da09bb2e45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["id"] = X_test.index


In [10]:
GBR = pd.DataFrame(y_pred,columns=["price"])
GBR.index.names=["id"]
GBR.to_csv("GradientBoostingRegressor.csv")

# ANALYZE RESULTS VIA METRICS

### First we prepare the Train_Test_Split and the predictions

In [11]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.2)

In [12]:
pred_train = model.predict(X_train1)
pred_test = model.predict(X_test1)

## CROSS VALUATION

In [13]:
from sklearn.model_selection import cross_val_score

In [14]:
cross = cross_val_score(model,X_train,y_train,cv = 20)
print(f"Cross Validation Result is {np.mean(cross)}")

Cross Validation Result is 0.9704228172942898


## R2

In [15]:
from sklearn.metrics import r2_score
train_r2 = r2_score(pred_train,y_train1)
print (f"R2 Train result is {train_r2}")
test_r2 = r2_score(pred_test,y_test1)
print(f"R2 Test result is {test_r2}")

R2 Train result is 0.9710177470266366
R2 Test result is 0.9709706649358035


## MSE

In [16]:
from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(pred_train,y_train1)
print(f"MSE Train result is {train_mse}")
test_mse = mean_squared_error(pred_test,y_test1)
print(f"MSE Train result is {test_mse}")

MSE Train result is 437233.71419588325
MSE Train result is 429969.40726848325


## MAE

In [17]:
from sklearn.metrics import mean_absolute_error
train_mae = mean_absolute_error(pred_train,y_train1)
print(f"MAE Train result is {train_mae}")
test_mae = mean_absolute_error(pred_test,y_test1)
print(f"MAE Test result is {test_mae}")

MAE Train result is 366.14738631546794
MAE Test result is 362.51751858627


## RMSE

In [18]:
from math import sqrt
train_rmse = sqrt(mean_absolute_error(y_train1,pred_train))
print(f"RMSE Train result is {train_rmse}")
test_rmse = sqrt(mean_absolute_error(y_test1,pred_test))
print(f"RMSE Test result is {test_rmse}")

RMSE Train result is 19.134978085053245
RMSE Test result is 19.039892819715924
